In [1]:
from bs4 import BeautifulSoup
import urllib
import re
import datetime
import pandas as pd
import numpy as np

In [2]:
def get_skills(tags, idd):
    #this function extracts the skill level of each player for each 
    #discipline from their Skill Levels page, and returns the skills as a list
    
    idx = [i for i, s in enumerate(tags[off:52]) if 'Ball' in str(s.text)]
    eightgames = str(tags[idx[1]+off].text).split(':')[1]
    eightskill = str(tags[idx[0]+off].text).split(':')[1]
    ninegames = str(tags[idx[5]+off].text).split(':')[1]    
    nineskill = str(tags[idx[4]+off].text).split(':')[1]
    tengames = str(tags[idx[8]+off].text).split(':')[1]    
    tenskill = str(tags[idx[7]+off].text).split(':')[1]
    feightgames = str(tags[idx[-1]+off+6].text).split(':')[1]    
    feightskill = str(tags[idx[-1]+off+2].text).split(':')[1]
    
    return [idd, eightgames, eightskill, ninegames, nineskill, tengames, tenskill, feightgames,
            feightskill]

In [11]:
player_skills = pd.DataFrame()
#read the existing database of players
ply = pd.read_csv('C:\\Users\\Owner\\Napa\\players.csv').dropna()   

#Player ID at which to begin search
pid1 = 53000
#Number of pages to scrape
n = 5

#player IDs to target 
player_ids = [int(i) for i in ply['ID'][pid1:pid1+n]]
#offset parameter; the first 21 tags don't contain relevant information
off = 21                 

for i, idd in enumerate(player_ids):
    
    add = 'https://www.napaleagues.com/stats.php?playerSelected=Y&playerID=' +str(idd) + '&xTab=20'
    r = urllib.urlopen(add).read()
    soup = BeautifulSoup(r, 'lxml')
    tags = soup.find_all('td')
    [tag.decompose() for tag in tags[off:52] if 'Class' in str(tag.text)] 
    #delete tags which contain 'Class' string, since the presence of these tags disrupts the position of the skill tags 
    player_skills[i]=get_skills(tags, idd)

#export dataframe to csv file
player_skills = player_skills.T
player_skills.columns = ['ID','8 Games','8 Skill','9 Games','9 Skill','10 Games','10 Skill','F8 Games','F8 SKill']

add = r'C:\\Users\\Owner\\Napa\\players_skills' + str(pid1) + '_' + str(n) +'.csv'
export_players=player_skills.to_csv(add)
player_skills.tail()

,ID,8 Games,8 Skill,9 Games,9 Skill,10 Games,10 Skill,F8 Games,F8 SKill
602,10063911,0,50,3,41,50,TBD,0,50
603,10063912,1,47,0,50,50,TBD,1,53
604,10063915,2,61,0,TBD,TBD,TBD,TBD,TBD
605,10063917,2,49,0,TBD,TBD,TBD,TBD,TBD
606,10063918,2,51,1,47,48,TBD,0,40


In [2]:
import glob
import csv

#combine all csv files into one

delimiter = ','
csvs = glob.glob('C:\\Users\\Owner\\Napa\\*.csv')
combined_csv = pd.concat([pd.read_csv(f) for f in csvs])
combined_csv.to_csv('C:\\Users\\Owner\\Napa\\combined_skills.csv')